### Preparação do ambiente

In [7]:
#!pip install -U requests lxml pandas requests

In [8]:
import pandas as pd
import requests
import webbrowser
from ipywidgets import interact, interact_manual

In [9]:
pd.options.display.float_format = '{:.2f}'.format
page_data = requests.get("https://www.fundsexplorer.com.br/ranking").text

### Leitura dos dados

In [10]:

df_fiis = pd.read_html(page_data)[0]

# remove os caracteres que irão impedir conversão de dados
df_fiis = df_fiis.replace(["R\$", "%", "\."], "", regex = True)

# substitui , por . (para permitir a transformação do string em float)
df_fiis = df_fiis.replace(",", ".", regex = True)

# valor do P/VPA muda durante a importacão, isso é necessário para retorná-lo ao valor original
df_fiis["P/VPA"] = df_fiis["P/VPA"] / 100
df_fiis["VPA"] = pd.to_numeric(df_fiis["VPA"], downcast="float") / 100


### Visualização

In [11]:
#df_fiis.head()
df_fiis.set_index('Código do fundo').loc[['CYCR11']]

,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) Média,DY (6M) Média,...,Patrimônio Líq.,VPA,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. no Período,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos
Código do fundo,,,,,,,,,,,,,,,,,,,,,
CYCR11,Títulos e Val Mob,8.40,51022.00,0.10,1.16,3.53,6.80,14.60,1.18,1.13,...,155109836.70,0.95,0.09,0.11,-0.69,-0.59,6.16,NaN,NaN,0


##### Detalhamento de algumas colunas

- `Preço atual`: valor da cota
- `Liquidez diária`: valor 
- `Dividendo`: valor pago para cada cota 
- `Dividend Yield`: Retorno calculado sobre a divisão do último dividendo anunciado pelo valor da cota no fechamento do mÊs
- `DY (XX) acumulado`- Soma do Dividend Yield nos últimos x períodos (podendo ser M-mês ou Y-ano)
- `DY (XX) média`- Média do Dividend Yield nos últimos x períodos (podendo ser M-mês ou Y-ano)
- `DY ano`- Soma do [Dividend Yield] no último ano
- `VPA`: resultado da divisão do Patrimônio líquido do fundo sobre o total de todas do fundo
- `P/VPA`: Indicador calculado pela divisão do preço da cota sobre o valor do patrimonial do fundo
 
 

### Lista de fundos que serão utilizados na classificação

##### Caso queira utilizar apenas fundos específicos

Descomente este bloco e preencha a lista com os fundos que deseja acompanhar

In [12]:
## fundos analistados
#fii_list = [
#    "IRDM11",    "BCFF11",    "BRCR11",    "BTLG11",    "HGCR11",    "VRTA11",    "GGRC11",
#    "HCTR11",    "FIIB11",    "KISU11",    "KNRI11",    "MXRF11",    "QAGR11",    "TORD11",
#    "XPLG11",    "HGRE11",    "PVBI11",    "BRCO11",    "XPML11",    "HGCR11",    "CPTS11",
#    "MCCI11"     "BTCR11",    "RBRR11",    "KNCR11",    "XPML11",    "VILG11",    "HSLG11",
#    "VISC11",    "RBRY11",    "BTRA11",    "TRXF11",    "CVBI11",]

#### Caso queira utizar todos os fundos disponíveis exceto alguns

Esta opcão é ideal para quem está iniciando e quer conhecer todos os fundos disponíveis. Uma vez selecionados os fundos desejados este bloco pode se comentado ou excluído

In [13]:
# todos os fundos disponíveis
fii_list = df_fiis['Código do fundo'].unique().tolist()

# fundos que devem ser removidos
fii_list.remove('GESE11B')
fii_list.remove('QIRI11')

### Calculo da nota do fundo

A nota é calculada multiplicando o:
 - Indicador calculado pela divisão do preço da cota sobre o valor do patrimonial do fundo
 - Média do Dividend Yield no último ano

Desta forma é possível identificar os fundos que podem gerar maior retorno neste período

In [14]:
filtered_fiis = df_fiis[df_fiis["Código do fundo"].isin(fii_list)]
selected_columns = filtered_fiis[["Código do fundo", "Setor", "Preço Atual", "Dividend Yield", "DY (3M) Média", "DY (6M) Média", "DY (12M) Média", "VPA", "P/VPA"]]

selected_columns['Nota'] = (
                                (1 / selected_columns['P/VPA']) 
                                * pd.to_numeric(df_fiis["DY (12M) Acumulado"], downcast="float")
                           )

selected_columns.sort_values(by=["Nota"], inplace=True, ascending=[False])

C:\Users\marco\AppData\Local\Temp\ipykernel_4316\804374413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['Nota'] = (
C:\Users\marco\AppData\Local\Temp\ipykernel_4316\804374413.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.sort_values(by=["Nota"], inplace=True, ascending=[False])


### Resultado

##### Top 10 Fundos Imobiliários

In [15]:
selected_columns.nlargest(10, 'Nota')

,Código do fundo,Setor,Preço Atual,Dividend Yield,DY (3M) Média,DY (6M) Média,DY (12M) Média,VPA,P/VPA,Nota
132,CYCR11,Títulos e Val Mob,8.40,1.16,1.18,1.13,1.22,0.95,0.09,162.22
41,IBFF11,Títulos e Val Mob,3.08,0.44,0.67,0.76,0.79,0.15,0.21,45.33
40,IBFF11,Títulos e Val Mob,3.08,0.44,0.67,0.76,0.79,0.15,0.21,45.33
9,FAMB11B,Lajes Corporativas,811.00,0.56,0.72,0.80,0.87,30.27,0.27,38.63
258,XPCM11,Lajes Corporativas,14.77,1.14,0.91,0.92,0.91,0.52,0.29,37.45
218,FPAB11,Lajes Corporativas,119.34,0.84,0.81,0.80,0.75,4.10,0.29,31.03
87,MBRF11,Outros,679.00,1.47,1.47,1.60,1.48,11.33,0.60,29.62
104,EDGA11,Lajes Corporativas,17.73,0.85,0.61,0.66,0.64,0.66,0.27,28.22
7,NEWU11,Lajes Corporativas,40.26,0.00,0.00,0.09,1.39,0.65,0.62,26.82
262,BBFI11B,Lajes Corporativas,1925.00,1.40,1.41,1.42,1.37,28.55,0.67,24.48


##### Salva dados em arquivo CSV

In [17]:
selected_columns.to_csv('fiis.csv')

### Consulta 

Visualização por fundo

Ao selecionar um fundo os dados disponíveis serão exibidos e seu navegador será aberto com mais detalhes

In [28]:
   
def list_fii_data(fundo):
    webbrowser.open_new('https://www.fundsexplorer.com.br/funds/' + fundo)
    return selected_columns.set_index('Código do fundo').loc[[fundo]]

interact(
    list_fii_data, 
    fundo= selected_columns.sort_values(by=["Código do fundo"])['Código do fundo'].unique(). tolist()
    
)

interactive(children=(Dropdown(description='fundo', options=('ABCP11', 'AFHI11', 'AFOF11', 'AGRX11', 'AIEC11',…

<function __main__.list_fii_data(fundo)>

Visualização por setor

In [22]:
def list_sector_data(sector):
    return selected_columns.set_index('Setor').loc[[sector]]

interact(
    list_sector_data, 
    sector = selected_columns['Setor'].unique().tolist()
)

interactive(children=(Dropdown(description='sector', options=('Títulos e Val Mob', 'Lajes Corporativas', 'Outr…

<function __main__.list_sector_data(sector)>